In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from dash import no_update
from jupyter_dash import JupyterDash

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
# spacex_df.head()

In [3]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [4]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
print(min_payload, max_payload)

0.0 9600.0


In [5]:
dele = spacex_df.groupby('Launch Site').get_group('CCAFS LC-40')
# dele.get_group('CCAFS LC-40')
dele['Launch Site'].unique()

array(['CCAFS LC-40'], dtype=object)

In [6]:
# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                
                                 dcc.Dropdown(id='site-dropdown', 
                                                     # Update dropdown values using list comphrehension
                                                     options=[
                                                         {'label': 'All Sites', 'value': 'ALL'},
                                                         {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                         {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    ],
                                                     value='ALL',
                                                     placeholder="All Sites",
                                                     # style={'width':'80%', 'padding':'2px', 'font-size': '15px', 'text-align-last' : 'left'},
                                                     searchable=True
                                                     ),  
                                
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=min_payload, max=max_payload, step=1000, 
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
  
    if entered_site == 'ALL':
        filtered_df = spacex_df[spacex_df['class']==1]
        fig = px.pie(filtered_df, values='class', names='Launch Site', 
        title='Total Success Launches by Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        # filtered_df = spacex_df.groupby('Launch Site').get_group(entered_site)
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        # print(filtered_df, type(filtered_df['class']))
        fig = px.pie(filtered_df, values=filtered_df['class'].value_counts().values, 
                     names=filtered_df['class'].value_counts().index,
                     title=f'Total Success Launches by Site {entered_site}')
        return fig                    
    
# *******************************************
# Task 3 and 4

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='payload-slider', component_property='value'), 
               Input(component_id='site-dropdown', component_property='value')]
             
             )

def get_scatter_chart(slider, entered_site):
    
    min_slider = slider[0]
    max_slider = slider[1]
    
    # print(slider, entered_site)
     
    if entered_site == 'ALL':
        payload_df = spacex_df[(spacex_df['Payload Mass (kg)']>=min_slider) & 
                           (spacex_df['Payload Mass (kg)']<=max_slider)]

    else:
        payload_df = spacex_df[(spacex_df['Payload Mass (kg)']>=min_slider) & 
                           (spacex_df['Payload Mass (kg)']<=max_slider) &
                           (spacex_df[spacex_df['Launch Site']==entered_site])]
                           
    scatter_fig = px.scatter(payload_df, x='Payload Mass (kg)', y='class', 
                             color='Booster Version Category', 
                             title='Correlation between Payload & Success for all Sites')
    
    return scatter_fig   

# Run the app
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)
